In [17]:
# Import necessary libraries

import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical operations
import pickle # For serializing and deserializing Python objects
from modelling_utils import train_knn_classifier, evaluate_model, plot_model_comparison, split_shuffle_data

In [18]:
# Load features from a pickle file
feature_dict_path = "/home/suraj/Repositories/FM-extractors-radiomics/evaluation/features/dlcs.pkl" # Path to the pickle file containing features
with open(feature_dict_path, 'rb') as file: # Open the file in read binary mode
    data = pickle.load(file) # Load the data from the pickle file

In [19]:
# Store test accuracies for each model
test_accuracies_dict = {} # Initialize an empty dictionary to store test accuracies

# Iterate through each model's features
for model_name, values in data.items(): # Loop through each model and its corresponding features
    # Skip MedImageInsightExtractor
    if model_name == "MedImageInsightExtractor": # Skip the MedImageInsightExtractor model
        continue

    # Extract paths and labels for train, val, and test sets
    train_labels = [v["row"]["Malignant_lbl"] for v in values["train"]] # Extract malignancy labels for the training set
    val_labels = [v["row"]["Malignant_lbl"] for v in values["val"]] # Extract malignancy labels for the validation set
    
    # Stack features
    train_items = np.vstack([v["feature"] for v in values["train"]]) # Stack features for the training set
    val_items = np.vstack([v["feature"] for v in values["val"]]) # Stack features for the validation set
    
    # Concatenate train and val
    all_items = np.vstack((train_items, val_items))
    all_labels = train_labels + val_labels

    # Split and shuffle data
    train_items, train_labels, val_items, val_labels, test_items, test_labels = split_shuffle_data(
        all_items, all_labels, train_ratio=0.5, val_ratio=0.2
    )

    # Train model with hyperparameter optimization
    best_model, study = train_knn_classifier(train_items, train_labels, val_items, val_labels)
    
    # Evaluate on test set
    test_accuracies_dict[model_name] = evaluate_model(best_model, test_items, test_labels)

[I 2025-01-22 16:27:52,366] A new study created in memory with name: no-name-ca864a21-ef41-4220-af68-82b44d266a48


[I 2025-01-22 16:27:52,444] Trial 0 finished with value: 0.5979670158697231 and parameters: {'k': 3}. Best is trial 0 with value: 0.5979670158697231.
[I 2025-01-22 16:27:52,484] Trial 1 finished with value: 0.664765065864537 and parameters: {'k': 29}. Best is trial 1 with value: 0.664765065864537.
[I 2025-01-22 16:27:52,531] Trial 2 finished with value: 0.6791826574006845 and parameters: {'k': 14}. Best is trial 2 with value: 0.6791826574006845.
[I 2025-01-22 16:27:52,584] Trial 3 finished with value: 0.6869619334093974 and parameters: {'k': 11}. Best is trial 3 with value: 0.6869619334093974.
[I 2025-01-22 16:27:52,635] Trial 4 finished with value: 0.6772637693185354 and parameters: {'k': 27}. Best is trial 3 with value: 0.6869619334093974.
[I 2025-01-22 16:27:52,715] Trial 5 finished with value: 0.6971268540607821 and parameters: {'k': 25}. Best is trial 5 with value: 0.6971268540607821.
[I 2025-01-22 16:27:52,821] Trial 6 finished with value: 0.6760709469971995 and parameters: {'k':

In [20]:
# Plot test accuracies
fig = plot_model_comparison(test_accuracies_dict)
fig.show() # Show the plot